## Inserting and Reading data from MySQL using Pandas

First let's start with a basic piece of code that fetches the data that we want to insert in the database. For our example, we will get the data about the Citibike stations, using the correspoding API call provided by the Citibike website:

In [1]:
import requests

In [2]:
# Let's get the data from the Citibike API

# This gives information for each station that remains stable over time
url_stations = 'https://gbfs.citibikenyc.com/gbfs/en/station_information.json'
# This gives the live status of all the stations (e.g., bikes available etc)
url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'

# We fetch for now just the time-invariant data
results = requests.get(url_stations).json() 

In [3]:
# We only need a subset of the data in the JSON returned by the Citibike API, so we keep only what we need
data = results["data"]["stations"]

In [4]:
len(data)

947

In [5]:
import pandas as pd
df = pd.DataFrame(data)
df.head(5)

,name,eightd_station_services,region_id,has_kiosk,electric_bike_surcharge_waiver,capacity,eightd_has_key_dispenser,station_type,station_id,rental_url,external_id,lon,short_name,rental_methods,lat
0,W 52 St & 11 Ave,[],71,True,False,55,False,classic,72,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,66db237e-0aca-11e7-82f6-3863bb44ef7c,-73.993929,6926.01,"[CREDITCARD, KEY]",40.767272
1,Franklin St & W Broadway,[],71,True,False,33,False,classic,79,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,66db269c-0aca-11e7-82f6-3863bb44ef7c,-74.006667,5430.08,"[CREDITCARD, KEY]",40.719116
2,St James Pl & Pearl St,[],71,True,False,27,False,classic,82,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,66db277a-0aca-11e7-82f6-3863bb44ef7c,-74.000165,5167.06,"[CREDITCARD, KEY]",40.711174
3,Atlantic Ave & Fort Greene Pl,[],71,True,False,62,False,classic,83,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,66db281e-0aca-11e7-82f6-3863bb44ef7c,-73.976323,4354.07,"[CREDITCARD, KEY]",40.683826
4,W 17 St & 8 Ave,[],71,True,False,50,False,classic,116,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,66db28b5-0aca-11e7-82f6-3863bb44ef7c,-74.001497,6148.02,"[CREDITCARD, KEY]",40.741776


In [6]:
# We drop the 'rental methods' columns, 
# as they contains multiple values and 
# we cannot insert lists in a database cell.
df.drop(['rental_methods','eightd_station_services'], axis='columns', inplace = True)
df.head(5)

,name,region_id,has_kiosk,electric_bike_surcharge_waiver,capacity,eightd_has_key_dispenser,station_type,station_id,rental_url,external_id,lon,short_name,lat
0,W 52 St & 11 Ave,71,True,False,55,False,classic,72,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,66db237e-0aca-11e7-82f6-3863bb44ef7c,-73.993929,6926.01,40.767272
1,Franklin St & W Broadway,71,True,False,33,False,classic,79,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,66db269c-0aca-11e7-82f6-3863bb44ef7c,-74.006667,5430.08,40.719116
2,St James Pl & Pearl St,71,True,False,27,False,classic,82,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,66db277a-0aca-11e7-82f6-3863bb44ef7c,-74.000165,5167.06,40.711174
3,Atlantic Ave & Fort Greene Pl,71,True,False,62,False,classic,83,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,66db281e-0aca-11e7-82f6-3863bb44ef7c,-73.976323,4354.07,40.683826
4,W 17 St & 8 Ave,71,True,False,50,False,classic,116,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,66db28b5-0aca-11e7-82f6-3863bb44ef7c,-74.001497,6148.02,40.741776


### Writing a Pandas Dataframe in a MySQL Table

Now we will connect to our MySQL server. We will use the SQLAlchemy library of Python.

If you do not have the library, you need to install it by typing in the shell:

In [7]:
!sudo apt-get install python3-mysqldb
!sudo pip3 install -U sqlalchemy sql_magic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-mysqldb is already the newest version (1.3.10-1build1).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.
Requirement already up-to-date: sqlalchemy in /usr/local/lib/python3.6/dist-packages (1.3.16)
Requirement already up-to-date: sql_magic in /usr/local/lib/python3.6/dist-packages (0.0.4)


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [8]:
import sqlalchemy
from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}/'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    password = 'dwdstudent2015')

engine = create_engine(conn_string)


Once we have connected successfully, we need to create our database:

In [9]:
# Query to create a database
# In this example, we will try to create the (existing) database "public"
# But in general, we can give any name to the database
db_name = 'public' 
create_db_query = f"CREATE DATABASE IF NOT EXISTS {db_name} DEFAULT CHARACTER SET 'utf8'"

# Create a database
engine.execute(create_db_query)

# And lets switch to the database
engine.execute(f"USE {db_name}")

In [10]:
# To avoid conflicts between people writing in the same database, we add a random suffix in the tables
# We only create the variable once while running the notebook
import uuid
if 'suffix' not in globals():
    suffix = str(uuid.uuid4())[:8]
print(suffix)

0730b7fe


### Create Table and Store Data in Database using the `to_sql` command

Then we create the table where we will store our data. Since we already have the data in a Pandas DataFrame, it is very easy to put the data in a database.

In [11]:
table_name = f'Stations_{suffix}'
# Create a table
# See http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html for the documentation

# This step is optional, but it is good practice to define explicitly the 
# data types before storing things in a database. In many cases, this can be ommitted, though.

dtype = {
    'station_id': sqlalchemy.types.SMALLINT(),
    'external_id': sqlalchemy.types.VARCHAR(50),
    'name': sqlalchemy.types.VARCHAR(50),
    'short_name': sqlalchemy.types.VARCHAR(10),
    'lat': sqlalchemy.types.Float, 
    'lon': sqlalchemy.types.Float,    
    'region_id': sqlalchemy.types.VARCHAR(5),
    'capacity': sqlalchemy.types.SMALLINT(),
    'rental_url': sqlalchemy.types.VARCHAR(100),
    'electric_bike_surcharge_waiver':  sqlalchemy.types.BOOLEAN,
    'eightd_has_key_dispenser':  sqlalchemy.types.BOOLEAN,    
    'has_kiosk':  sqlalchemy.types.BOOLEAN
}


df.to_sql(name = table_name, 
          schema = db_name, 
          con = engine, 
          if_exists = 'replace', 
          index = False, 
          dtype = dtype)

In [12]:
# Once we have the data in the table, we also specify a primary key
# If we had FOREIGN KEYS we can add them in the same way
add_key_query = f'ALTER TABLE {table_name} ADD PRIMARY KEY(station_id)'
print(add_key_query)
engine.execute(add_key_query)

ALTER TABLE Stations_0730b7fe ADD PRIMARY KEY(station_id)


### Reading from a SQL Database in Python using the `read_sql` command in Pandas

We can similarly read from the database using Pandas

In [13]:
query =  f"SELECT * FROM {db_name}.{table_name}"
print(query)

SELECT * FROM public.Stations_0730b7fe


In [14]:
df2 = pd.read_sql(query, con=engine)
df2.head(5)

,name,region_id,has_kiosk,electric_bike_surcharge_waiver,capacity,eightd_has_key_dispenser,station_type,station_id,rental_url,external_id,lon,short_name,lat
0,W 52 St & 11 Ave,71,1,0,55,0,classic,72,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,66db237e-0aca-11e7-82f6-3863bb44ef7c,-73.9939,6926.01,40.7673
1,Franklin St & W Broadway,71,1,0,33,0,classic,79,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,66db269c-0aca-11e7-82f6-3863bb44ef7c,-74.0067,5430.08,40.7191
2,St James Pl & Pearl St,71,1,0,27,0,classic,82,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,66db277a-0aca-11e7-82f6-3863bb44ef7c,-74.0002,5167.06,40.7112
3,Atlantic Ave & Fort Greene Pl,71,1,0,62,0,classic,83,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,66db281e-0aca-11e7-82f6-3863bb44ef7c,-73.9763,4354.07,40.6838
4,W 17 St & 8 Ave,71,1,0,50,0,classic,116,http://app.citibikenyc.com/S6Lr/IBV092JufD?sta...,66db28b5-0aca-11e7-82f6-3863bb44ef7c,-74.0015,6148.02,40.7418


### Export Data from Database to CSV or Excel

And remember that from Pandas it is also possible to export in other formats, such as Excel of CSV.

In [15]:
# The necessary library to write in Excel
!sudo pip3 install -U xlwt

Requirement already up-to-date: xlwt in /usr/local/lib/python3.6/dist-packages (1.3.0)
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [16]:
df2.to_excel("citibike.xls")
df2.to_csv("citibike.csv")

### Cleanup

Finally, let's clean up and delete the table that we created

In [17]:
drop_table_query = f"DROP TABLE IF EXISTS {db_name}.{table_name}"
print(drop_table_query)
engine.execute(drop_table_query)

DROP TABLE IF EXISTS public.Stations_0730b7fe


### Exercise

The `url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'` URL contains the status of the stations. Write code that reads the results from that API call, and then stores the data in a separate table. Add a "foreign key" constraint from the Status table to the Stations table that we created above.